In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree

In [2]:
def read_cloud(filename):
    ''' Read in pointcloud from .pcd file using pyntcloud module, 
    extract points as pandas dataframe and return dataframe only. 
    
    Reference:
    pyntcloud - Python PointCloud module
    © Copyright HAKUNA MATATA
    Project page: https://pyntcloud.readthedocs.io/en/latest/index.html'''
    
    import pyntcloud as pc
    return pc.PyntCloud.from_file(filename).points

def drop_null_pts(df, reset_index = True):
    ''' Filter out rows that contain NaN or are (0,0,0) 
    (per Piazza discussion, these "represent points too far away or too close") '''
    df = df.dropna(how = 'all') 
    df = df.drop(df[(df['x'] == 0) & (df['y'] == 0.0) & (df['z'] == 0.0)].index)
    
    if reset_index:
        df = df.reset_index(drop=True) # Re-number points with null points removed
    
    return df

In [3]:
def extract_plane(pts):
    ''' Fit a plane to a numpy array of points with PCA '''
    # Find and subtract out the centroid
    centroid = np.mean(pts,axis=0)
    pts -= centroid

    # Find the covariance matrix for the points
    cov = np.cov(pts,rowvar=False)

    # Find eigen-decomposition of cov matrix
    eigenvals, eigenvecs = np.linalg.eig(cov)

    # Make sure they are sorted from highest eigenvalue to lowest
    order = eigenvals.argsort()[::-1]
    eigenvals = eigenvals[order]
    eigenvecs = eigenvecs[:,order]
    
    # 3rd eigenvector is normal to the plane
    normal = eigenvecs[:,2]
    
    return normal, centroid


In [4]:
def calculate_SSE(cloud, normal, p):
    ''' Determine sum of squared error of the point cloud w.r.t. the plane. 
    
    The Squared Error of a point to the estimated plane is    
                                    (N.x - N.p)^2
    where . is the dot product, N = eigenvecs[:,-1] (the normal to the estimated plane), 
    and p = point on the plane.
    '''
    # Point-wise squared error
    SE = (cloud.dot(normal) - np.dot(normal,p) ) ** 2 
    # Sum squared error 
    SSE = SE.sum()
    return SSE



In [34]:
### my RANSAC method for finding the dominant plane in the cloud

def myRANSAC(cloud, k=None, n_iters=None, return_SSE=False, return_stability=False):
    ''' Use a form of RANSAC to find the dominant plane in a pointcloud.
    
    Parameters
    ---------------
    cloud            -  Pointcloud, Pandas dataframe of Nx3 points
    k                -  k for k-nearest neighbors tree
    n_iters          -  maximum iterations before returning the best estimate plane
    return_SSE       -  flag to return Sum of Squared Error of plane over the whole cloud
    return_stability -  a measure of how many iterations the best estimate plane 
                        persisted for
    
    '''
    
    # If no k, use rule-of-thumb k=sqrt(num_points)
    if k is None:
        k = int(np.sqrt(len(cloud)))
    
    # Build a nearest-neighbors tree
    tree = KDTree(cloud,leafsize=k)
    
    # Initialize sum of squared errors
    best_SSE = float('inf')
    best_normal = None
    best_centroid = None
    
    # Get samples as numpy array
    if n_iters is None:
        # If no n_iters input, do the whole cloud
        sample = cloud.to_numpy()
    else:
        # Otherwise randomly sample n_samples points from cloud
        sample = cloud.sample(n=n_iters).to_numpy()
    
    
    # For each sample point, estimate local plane and estimate how well 
    # it approximates the whole cloud
    for pt in sample:
        # Find k nearest neighbors
        dists, indices = tree.query(pt,k=k)
        NN = cloud.iloc[indices].to_numpy()
        
        # Extract best-fitting plane with PCA as a normal vector and point on plane
        normal, centroid = extract_plane(NN)
        
        # Calculate support for this plane within entire cloud as SSE
        SSE = calculate_SSE(cloud,normal,centroid)
        
        # Compare SSE with previous best estimate and update estimate if better
        if SSE < best_SSE:
            best_normal = normal
            best_centroid = centroid
            best_SSE = SSE
            
            # Measure how long a best estimate has stuck around
            stability = 0
        else:
            stability += 1
        
        
    
    # Check if final plane is oriented towards camera -> flip if not 
    # Assume camera is at (0,0,0), so we want the normal to be the 
    # opposite direction of the offset of the plane (i.e. the centroid).
    if np.dot(normal,centroid) > 0:
        normal *= -1
    
    # Collect requested return values
    RETURN = [best_normal, best_centroid]
        
    if return_SSE:
        RETURN.append(best_SSE)
    if return_stability:
        RETURN.append(stability/len(sample))

    return RETURN


In [42]:
# Filenames
empty = 'Empty.pcd'
table = 'TableWithObjects.pcd'
hallway = 'Hallway1a.pcd'

# Import pointcloud
cloud = read_cloud(empty)

# Filter out rows that don't contain any data 
cloud = drop_null_pts(cloud)



In [43]:
plane = myRANSAC(cloud, k=10, n_iters=1000, return_SSE=True, return_stability=True)
plane

[array([0.09423926, 0.85064469, 0.51722584]),
 array([-0.5179812 ,  0.27938896,  1.2729    ], dtype=float32),
 6990.373675563548,
 0.742]

In [32]:
np.dot(plane[0],plane[1])

-1.1278005166936367

In [8]:
# N = len(cloud) # total point count
# max_k = 25 #int(np.sqrt(N)*2) # max k value to test
# min_k = 10
# num_k = 10

# # Test various values of k
# for k in np.linspace(min_k, max_k, num_k, dtype=int):
#     _,_,SSE = myRANSAC(cloud, k=k, n_iters=1000, return_SSE=True)
#     print(k,SSE)
    

In [40]:
# Finding dominant planes in the hallway
# Strategy: Find first dominant plane, catalog, remove "supporting points", 
#           repeat until end condition

def pts_on_plane(cloud,normal,centroid,delta=None,kdelta=0.1):
    # Point-wise squared error
    SE = (cloud.dot(normal) - np.dot(normal,centroid) ) ** 2 
    # Total squared error 
    SSE = SE.sum()
    
    # If unspecified, delta (the cutoff for whether a point is 
    # "close enough" to the plane) will be set to kdelta times the
    # average error
    if delta is None:     
        delta = np.sqrt(SSE)/len(cloud) * kdelta

    # Collect indices of points within delta of estimated plane
    ERR = np.sqrt(SE)
    inds = ERR[ERR < delta].index.tolist()

    return inds


def my_extended_RANSAC(cloud, k=None, n_iters=None, return_SSE=False, return_stability=False):
    
    planes = []
    MAX_PLANES = 3
    min_size = int(len(cloud) * 0.05)
        
    while(len(planes)<MAX_PLANES and len(cloud) > min_size):
        # Run myRANSAC to get dominant plane
        out = myRANSAC(cloud, k=k, n_iters=n_iters, return_SSE=return_SSE, return_stability=return_stability)
        planes.append(out)
        
        # Find supported points and drop them
        pts = pts_on_plane(cloud,out[0],out[1], kdelta=0.25)
        cloud = cloud.drop(pts)
    
    return planes
    
    
    
    

In [41]:
my_extended_RANSAC(cloud, k=10, n_iters=50, return_SSE=True, return_stability=True)

21
7
74


[[array([-0.03992603,  0.81915883,  0.57217543]),
  array([ 0.13874812, -0.14890206,  1.7126999 ], dtype=float32),
  550.9962594406802,
  0.58],
 [array([0.05949353, 0.83783288, 0.54267539]),
  array([-0.8548542 , -0.39532846,  2.2356    ], dtype=float32),
  490.6116268481861,
  0.06],
 [array([0.01169389, 0.88082039, 0.47330613]),
  array([0.28489506, 0.0190324 , 1.3528    ], dtype=float32),
  625.3867579577343,
  0.9]]

In [20]:
C = cloud.dot(np.array([0,1,2]))

C[C<3].index.tolist()

[26593,
 26594,
 26993,
 26994,
 27399,
 27400,
 27814,
 27815,
 27816,
 28258,
 28259,
 90410,
 91004,
 91005,
 91006,
 91206,
 91207,
 91208,
 91209,
 91210,
 91211,
 91237,
 91238,
 91239,
 91599,
 91600,
 91601,
 91602,
 91603,
 91604,
 91801,
 91802,
 91803,
 91804,
 91805,
 91806,
 91807,
 91808,
 91811,
 91812,
 91813,
 91814,
 91815,
 91816,
 91817,
 91818,
 91819,
 91820,
 91821,
 91822,
 91823,
 91824,
 91827,
 91828,
 91829,
 91830,
 91831,
 91832,
 91833,
 91834,
 92124,
 92125,
 92126,
 92127,
 92128,
 92130,
 92131,
 92147,
 92148,
 92149,
 92150,
 92151,
 92152,
 92153,
 92154,
 92155,
 92156,
 92157,
 92168,
 92169,
 92170,
 92171,
 92172,
 92173,
 92174,
 92175,
 92179,
 92180,
 92181,
 92182,
 92183,
 92184,
 92185,
 92186,
 92187,
 92188,
 92189,
 92190,
 92191,
 92192,
 92193,
 92194,
 92195,
 92196,
 92197,
 92198,
 92199,
 92200,
 92395,
 92396,
 92397,
 92398,
 92399,
 92400,
 92401,
 92402,
 92403,
 92404,
 92410,
 92411,
 92412,
 92413,
 92415,
 92416,
 92417,


In [28]:
planes[0][0]

NameError: name 'planes' is not defined